# Baseline Direct Translation Pipeline (Marian + NLLB)
### Honors Thesis – Fall 2025 Milestone  
Author: Jameela Kauser  
Goal: Implement direct translation baseline and evaluation protocol.


1. Install Dependencies

In [1]:
!pip install transformers sentencepiece sacrebleu evaluate bert-score comet-ml pandas numpy tqdm gitpython

2. Imports & GPU setup

In [2]:
import torch
import pandas as pd
import numpy as np
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
import sacrebleu
import evaluate
from bert_score import score as bertscore
from tqdm import tqdm
import os

device = "cuda" if torch.cuda.is_available() else "cpu"
device


'cpu'

3. Load Marian + NLLB

In [3]:
marian_translator = pipeline(
    "translation",
    model="Helsinki-NLP/opus-mt-en-fr"
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
Device set to use cpu


In [4]:
nllb_model_name = "facebook/nllb-200-distilled-600M"

nllb_tokenizer = AutoTokenizer.from_pretrained(nllb_model_name)
nllb_model = AutoModelForSeq2SeqLM.from_pretrained(nllb_model_name).to(device)

def nllb_translate(text, tgt="fra_Latn"):
    inputs = nllb_tokenizer(text, return_tensors="pt").to(device)
    bos = nllb_tokenizer.convert_tokens_to_ids(tgt)
    output = nllb_model.generate(
        **inputs,
        forced_bos_token_id=bos
    )
    return nllb_tokenizer.decode(output[0], skip_special_tokens=True)

4. Run baseline translations

In [5]:
def run_translations(sentences, tgt_lang="fra_Latn", out_path="baseline_outputs.csv"):
    logs = []
    for s in tqdm(sentences):
        marian_out = marian_translator(s)[0]['translation_text']
        nllb_out = nllb_translate(s, tgt=tgt_lang)

        logs.append({
            "source": s,
            "marian_translation": marian_out,
            "nllb_translation": nllb_out
        })

    df = pd.DataFrame(logs)
    df.to_csv(out_path, index=False)
    return df

In [6]:
sentences = [
    "Knowledge is power.",
    "Education should be accessible to everyone.",
    "Language connects people.",
    "This is a test sentence for evaluation."
]

df = run_translations(sentences, tgt_lang="fra_Latn")
df


100%|██████████| 4/4 [00:11<00:00,  2.94s/it]


,source,marian_translation,nllb_translation
0,Knowledge is power.,"La connaissance, c'est le pouvoir.",Le savoir est le pouvoir.
1,Education should be accessible to everyone.,L'éducation devrait être accessible à tous.,L'éducation devrait être accessible à tous.
2,Language connects people.,La langue relie les gens.,Le langage relie les gens.
3,This is a test sentence for evaluation.,Il s'agit d'une phrase d'essai pour l'évaluation.,C'est une phrase d'essai pour évaluation.


5. BLEU and BERTScore evaluation

In [7]:
def compute_bleu(hypotheses, references):
    return sacrebleu.corpus_bleu(hypotheses, [references]).score

In [8]:
def compute_bertscore(hypotheses, references, lang="fr"):
    _, _, F1 = bertscore(hypotheses, references, lang=lang)
    return float(F1.mean())

In [9]:
df["reference"] = df["nllb_translation"]  # Placeholder until real references

6. Run Evaluation on Both Models

In [10]:
marian_hyp = df["marian_translation"].tolist()
nllb_hyp = df["nllb_translation"].tolist()
ref = df["reference"].tolist()
src = df["source"].tolist()

results = {
    "BLEU_marian": compute_bleu(marian_hyp, ref),
    "BLEU_nllb": compute_bleu(nllb_hyp, ref),
    "BERT_marian": compute_bertscore(marian_hyp, ref),
    "BERT_nllb": compute_bertscore(nllb_hyp, ref)
}

results

{'BLEU_marian': 47.041079561594984,
 'BLEU_nllb': 100.00000000000004,
 'BERT_marian': 0.9446861743927002,
 'BERT_nllb': 1.0}

7. Save Metrics to CSV

In [11]:
pd.DataFrame([results]).to_csv("baseline_metrics.csv", index=False)